# Caso do Churrasco

Neste notebook o nosso objetivo é trabalhar modelagem e algoritmos de otimização através do problema prático do churrasco. Este problema parte da necessidade de comprar itens para realizar o churrasco de fim de ano, porém temos uma grande limitação: orçamento. Desta forma, queremos encontrar quais itens compramos de modo a utilizarmos a maior parte do dinheiro, obter a maior quantidade de alimentos e também atingir uma boa diversidade.

Como fica evidente, este problema é uma otimização multiobjetivo. Nossa função fitness (ou função objetivo) precisará lidar com esta particularidade e, em especial, permitir alterarmos a forma de compilar cada objetivo de forma isolada durante a execução dos algoritmos.

Trataremos neste notebook a concepção do problema, a modelagem, configuração, testes e por fim algoritmos de otimização. Resumidamente, os conceitos abordados serão:

- Organização de parâmetros de configuração de entrada do problema;
- Codificação de modelo para o problema utilizando OO;
- Formas de utilizar diversos objetivos para avaliar soluções;
- Implementar busca local com Algoritmo da Subida de Encosta;
- Solucionar problema com Algoritmo de Têmpera Simulada;
- Criar Algoritmo Genético para solucionar o problema;
- Adicionar etapa de explotação com busca local para aprimorar Algoritmo Genético

# Configuração de Ambiente

Nesta seção configuraremos todas as bibliotecas necessárias, bem como os caminhos para arquivos que serão utilizados. Este notebook faz parte de um repositório maior, portanto foi desenvolvido para trabalhar sinergicamente com os demais diretórios, em especial o de armazenamento de dados, onde está o arquivo de itens disponíveis para a compra. Caso deseje executar localmente, atente-se apenas para manter os diretórios do repositório original sem alteração, de modo que os comandos abaixo consigam encontrar os caminhos desejados.

Vamos trabalhar também com uma biblioteca auxiliar de tipagem, com o intuito de deixar mais claro o que são as variáveis trabalhadas em cada parte do código, sobretudo os argumentos de construtores e métodos. Desta forma, nosso código ficará mais legível e, mesmo depois de algum tempo sem nos debruçar sobre o código, conseguiremos entender o que cada variável significa. Outra grande funcionalidade é a possibilidade de utilizar o intellisense do editor de texto, o que é de grande ajuda quando estamos escrevendo código.

Por fim, utilizaremos o inglês para nomear variáveis. Esta é uma decisão puramente estética pessoal, para evitar a escrita "errada" de palavras em português quando precisarmos remover acentuações e caracteres especiais. Saiba que esta prática não representa nenhum tipo de convenção ou melhoria de performance, apenas se trata de uma forma de escrita pessoal.

In [1]:
import os
import json
import random
from enum import Enum
from typing import List, Tuple, Callable

BASE_PATH = os.path.dirname(os.getcwd())
ITEMS_PATH = f"{BASE_PATH}/data/churrasco.json"

# Criação do Modelo

O ponto de partida para começar a codificar é definir, textualmente, os requisitos que nosso modelo precisa entregar. Esta etapa serve como bússula para quando formos escrever o código, evitando que fiquemos travados diante da tela. Assim, são requisitos do modelo:

- Configurar e processar os itens que podem ser comprados;
- Registar informação de limite do orçamento disponível (restrição);
- Forma de representar a solução do problema, ou seja, itens a serem comprados;
- Procedimento de avaliar validade de uma solução, confrontando com o orçamento;
- Estratégia para encontrar soluções vizinhas, dada uma solução qualquer;
- Método de avaliação da solução a partir de regras específicas (função fitness).

## Itens disponíveis

Para configurar os itens de entradas utilizaremos um arquivo JSON como exemplificado abaixo. A estratégia deste JSON é configurar itens de forma genérica, introduzindo o conceito de compra por lote. Carnes geralmente são compradas por kg, então sua "Unidades/Lote" possui valor *null*. Por outro lado, o pão de alho é comprado por lotes de 10 unidades. O campo "Incremento" informa de quanto em quanto um item pode ser comprado. Novamente, carnes poderiam ser compradas em qualquer quantidade, porém não é prático pedir frações menores que 100g. Por outro lado, o pão de alho sempre é comprado de lote em lote, ou seja de 1 em 1. Os campos "Preço/Lote" e "kg/Lote" são autoexplicativos, indicam o preço pago por um lote do item (para as carnes um lote é 1kg e para o pão de alho o lote são 10 pães) e quanto de massa o lote tem, respectivamente.
```
{
    "Alcatra": {
        "Preço/Lote": 37.99,
        "Unidades/Lote": null,
        "kg/Lote": 1,
        "Incremento": 0.1
    },
    "Pão de Alho": {
        "Preço/Lote": 15.97,
        "Unidades/Lote": 10,
        "kg/Lote": 0.8,
        "Incremento": 1
    },
    "Picanha": {...},
    "Pernil": {...},
    ...
}
```

Com o JSON estruturado, vamos criar a classe responsável por transcrever esta estrutura de dados. Esta será a classe utilizada pelo modelo para gerenciar os itens disponíveis para compra, evitando trabalhar com strings de chave e valor, acessando o JSON (transformado em dicionário no python.).

In [1]:
class Item:
    def __init__(self, name: str, batch_price: float,
        batch_quantity: float | None, increment: float,
        batch_kg: float
    ) -> None:
        self.name = name
        self.batch_kg = batch_kg
        self.increment = increment
        self.batch_price = batch_price
        self.batch_quantity = batch_quantity

In [125]:
class ItemCategory(Enum):
    CARNE_BOVINA = 1
    CARNE_SUINA = 2
    CARNE_AVES = 3
    OUTROS = 4


class Item:
    def __init__(self, name: str, batch_price: float,
        batch_quantity: float | None, increment: float,
        batch_kg: float, category: ItemCategory
    ) -> None:
        self.name = name
        self.category = category
        self.batch_kg = batch_kg
        self.increment = increment
        self.batch_price = batch_price
        self.batch_quantity = batch_quantity


class Objective(Enum):
    MASSA_TOTAL = 1
    GASTO_TOTAL = 2
    DIVERSIDADE = 3


class Barbecue:
    def __init__(self, money_limit: float) -> None:
        self.items: List[Item] = []
        self.mass_limit: float = None
        self.money_limit = money_limit
        self.relate_objective = {
            Objective.MASSA_TOTAL: self.calculate_mass,
            Objective.GASTO_TOTAL: self.calculate_spent,
            Objective.DIVERSIDADE: self.calculate_diversity
        }

    def random_solution(self) -> List[float]:
        while True:
            solution: List[float] = []
            for i in range(0, len(self.items)):
                multiply = random.randint(0, 10)
                quantity = round(self.items[i].increment * multiply, 1)
                solution.append(quantity)

            if self.valid_solution(solution):
                break

        return solution

    def near_solution(self, solution: List[float], quantity: int) -> List[List[float]]:
        new_solutions: List[List[float]] = []
        
        for _ in range(0, quantity):
            while True:
                new_solution = solution.copy()
                i = random.randint(0, len(solution) - 1)
                signal = 1 if random.random() < 0.5 else -1
                new_solution[i] = round(new_solution[i] + signal * self.items[i].increment, 1)

                if self.valid_solution(new_solution):
                    break

            new_solutions.append(new_solution)
        
        return new_solutions

    def calculate_fitness(self, solution: List[float], compose: List[Tuple[Objective, float]]) -> float:
        score = 0
        for objective, weight in compose:
            fitness = self.relate_objective[objective](solution)
            score = score + fitness * weight
        return score

    def calculate_spent(self, solution: List[float]) -> float:
        spent = 0
        for i in range(0, len(solution)):
            spent = spent + self.items[i].batch_price * solution[i]
        return spent / self.money_limit
    
    def calculate_mass(self, solution: List[float]) -> float:
        mass = 0
        for i in range(0, len(solution)):
            mass = mass + self.items[i].batch_kg * solution[i]
        return mass / self.mass_limit
    
    def calculate_diversity(self, solution: List[float]) -> float:
        # Encontra a massa de cada item
        mass = []
        for i in range(0, len(solution)):
            mass.append(self.items[i].batch_kg * solution[i])
        
        # Calcula a fração mássica de cada item
        mass_total = sum(mass)
        frac = [m / mass_total for m in mass]

        # Define diversidade como a razão entre a menor fração
        # mássica e a fração mássica em condições totalmente
        # iguais para todos os itens
        equals_frac = 1 / len(solution)
        return min(frac) / equals_frac

    def process_json(self, data: dict) -> None:
        # Carrega itens do json
        items: List[Item] = []
        for item in data.keys():
            items.append(Item(
                item,
                data[item]["Preço/Lote"],
                data[item]["Quantidade/Lote"],
                data[item]["Incremento"],
                data[item]["Massa/Lote"],
                self.__format_category(data[item]["Categoria"])
            ))
        self.items = items

        # Calcula a massa máxima que pode ser obtida
        max_mass = 0
        for item in self.items:
            spent, multiply = 0, 0
            while spent < self.money_limit:
                spent = spent + item.increment * item.batch_price
                multiply = multiply + item.increment

            multiply = multiply - item.increment
            item_mass = multiply * item.batch_kg
            if item_mass > max_mass:
                max_mass = item_mass

        self.mass_limit = max_mass

    def valid_solution(self, solution: List[float]) -> bool:
        spent = 0
        for i in range(0, len(solution)):
            spent = spent + self.items[i].batch_price * solution[i]

        if spent > self.money_limit:
            return False
        else:
            return True
        
    def print_solution(self, solution: List[float]) -> None:
        print("----Apresentando resultados da solução----")
        for i in range(0, len(solution)):
            spent = self.items[i].batch_price * solution[i]
            mass = self.items[i].batch_kg * solution[i]
            print(f"{self.items[i].name}: R$ {round(spent, 2)} ({round(mass, 1)} kg)")

    def __format_category(self, category: str) -> ItemCategory:
        if category == "Carne Bovina":
            return ItemCategory.CARNE_BOVINA
        elif category == "Carne Suína":
            return ItemCategory.CARNE_SUINA
        elif category == "Carne de Aves":
            return ItemCategory.CARNE_AVES
        else:
            return ItemCategory.OUTROS

In [130]:
with open(ITEMS_PATH) as file:
    data = json.load(file)

barbecue = Barbecue(money_limit=300)
barbecue.process_json(data)

solution = barbecue.random_solution()
print(solution)

fitness_spent = barbecue.calculate_spent(solution)
print(f"Fitness de gasto: {fitness_spent}")

fitness_mass = barbecue.calculate_mass(solution)
print(f"Fitness de massa: {fitness_mass}")

fitness_diversity = barbecue.calculate_diversity(solution)
print(f"Fitness de diversidade: {fitness_diversity}")

compose = [
    (Objective.MASSA_TOTAL, 0.3),
    (Objective.GASTO_TOTAL, 0.5),
    (Objective.DIVERSIDADE, 0.2)
]
fitness_total = barbecue.calculate_fitness(solution, compose)
print(f"Fitness geral: {fitness_total}")

barbecue.print_solution(solution)

neighbors = barbecue.near_solution(solution, quantity=10)
print("Vizinhos:")
for n in neighbors:
    print(n)

[0.0, 0.8, 0.5, 1.0, 0.7, 3]
Fitness de gasto: 0.5396000000000001
Fitness de massa: 0.375
Fitness de diversidade: 0.0
Fitness geral: 0.38230000000000003
----Apresentando resultados da solução----
Alcatra: R$ 0.0 (0.0 kg)
Linguiça: R$ 30.39 (0.8 kg)
Pernil: R$ 19.0 (0.5 kg)
Lombo: R$ 37.99 (1.0 kg)
Picanha: R$ 26.59 (0.7 kg)
Pão de Alho: R$ 47.91 (2.4 kg)
Vizinhos:
[0.0, 0.8, 0.5, 1.0, 0.8, 3]
[0.0, 0.8, 0.5, 1.0, 0.6, 3]
[0.0, 0.8, 0.5, 0.9, 0.7, 3]
[0.0, 0.8, 0.4, 1.0, 0.7, 3]
[0.0, 0.8, 0.5, 1.0, 0.7, 2]
[0.0, 0.8, 0.5, 1.0, 0.8, 3]
[0.0, 0.8, 0.5, 0.9, 0.7, 3]
[0.0, 0.8, 0.5, 1.0, 0.6, 3]
[0.1, 0.8, 0.5, 1.0, 0.7, 3]
[0.0, 0.8, 0.5, 1.0, 0.8, 3]


In [123]:
(0.5801700000000001 * 0.5) + (0.42361111111111105 * 0.3) + (0.09836065573770493 * 0.2)

0.4368404644808743